# 빠른 시작 가이드 - Azure AI Foundry

이 노트북은 Azure AI Foundry에 대한 실습 소개를 제공합니다:
1. AI 프로젝트 클라이언트 초기화
2. 사용 가능한 모델 나열
3. 간단한 채팅 완료 요청을 생성
4. 기본 AI 에이전트 생성
5. 기본 오류 시나리오 다루기

## 전제 조건
- 이전 notebook에서 환경 설정 완료
- Azure 자격 증명 구성

## 필수 라이브러리 및 설정 가져오기

다음 코드의 내용은 다음을 수행합니다:
1. 인증 및 AI 프로젝트에 필요한 Azure SDK 라이브러리를 가져옵니다
2. 환경 변수 및 JSON 처리를 위한 표준 Python 라이브러리를 가져옵니다
3. DefaultAzureCredential을 사용하여 Azure 자격 증명을 초기화
   - 로그인한 Azure CLI 자격 증명을 자동으로 사용
   - 또는 환경 변수 또는 매니지드 아이덴티티와 같은 다른 인증 방법을 사용할 수 있습니다.


In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials
credential = DefaultAzureCredential()

## AI Project Client 초기화

> **참고:** 계속 진행하기 전에 다음 사항을 확인하세요:
> 1. 루트 디렉토리의 `.env.example` 파일을 `.env` 로 복사합니다.
> 2. `.env` 파일에서 프로젝트 연결 문자열을 업데이트합니다.
> 3. Azure AI Foundry에 허브 및 프로젝트를 프로비전하여 배포해서 가지고 있어야 합니다.

프로젝트의 설정 아래에서 [Azure AI Foundry](https://ai.azure.com)프로젝트 연결 문자열을 찾을 수 있습니다:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## AI Project Client 생성하기

다음 코드에서는 `.env` 파일의 연결 문자열을 사용하여 AI 프로젝트 클라이언트를 만들겠습니다.
> **Note:** This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing `asyncio` and using the async methods from `AIProjectClient`.

The client will be used to:
- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models


In [ ]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

try:
    client = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## Create a Simple Completion
Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request.
The code below demonstrates how to:
1. Get a ChatCompletionsClient from the azure-ai-inference package
2. Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our `.env` file, making it easy to switch between different
deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers
that support chat completions.

> Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in [README.md](../README.md#-quick-start))


In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    chat_client = client.inference.get_chat_completions_client()
    response = chat_client.complete(
        model=model_deployment_name, 
        messages=[UserMessage(content="How to be healthy in one sentence?")]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {str(e)}")

## Create a simple Agent

Using AI Agent Service, we can create a simple agent to answer health related questions.

Let's explore Azure AI Agent Service, a powerful tool for building intelligent agents.

Azure AI Agent Service is a fully managed service that helps developers build, deploy, and scale AI agents
without managing infrastructure. It combines large language models with tools that allow agents to:
- Answer questions using RAG (Retrieval Augmented Generation)
- Perform actions through tool calling 
- Automate complex workflows

The code below demonstrates how to:
1. Create an agent with a code interpreter tool
2. Create a conversation thread
3. Send a message requesting BMI analysis 
4. Process the request and get results
5. Save any generated visualizations

The agent will use the model specified in our .env file (MODEL_DEPLOYMENT_NAME) and will have access
to a code interpreter tool for creating visualizations. This showcases how agents can combine
natural language understanding with computational capabilities.

> The visualization will be saved as a PNG file in the same folder as this notebook.
 



In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

try:
    # Initialize the Code Interpreter Tool
    code_interpreter = CodeInterpreterTool()
    
    # Create an AI agent with the code interpreter tool
    agent = client.agents.create_agent(
        model=model_deployment_name,
        name="bmi-calculator",
        instructions=(
            "You are a health analyst who calculates BMI using US metrics (pounds, feet/inches). "
            "Use average US female measurements: 5'4\" (69 inches) and 130 pounds. "
            "Create a visualization showing where this BMI falls on the scale."
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    
    # Create a new conversation thread
    thread = client.agents.create_thread()
    
    # Create a message requesting BMI analysis and visualization
    message = client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            "Calculate BMI for an average US female (5'4\", 130 lbs). "
            "Create a visualization showing where this BMI falls on the standard BMI scale from 15 to 35. "
            "Include the standard BMI categories (Underweight, Normal, Overweight, Obese) in the visualization."
        )
    )
    
    # Process the request by creating and running a thread run
    run = client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    
    # Retrieve and save any generated visualizations
    messages = client.agents.list_messages(thread_id=thread.id)
    for image_content in messages.image_contents:
        file_name = f"bmi_analysis_{image_content.image_file.file_id}.png"
        client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Analysis saved as: {file_name}")
    
    # Print the analysis text from the assistant
    print(f"Messages: {messages}")
    if last_msg := messages.get_last_text_message_by_role("assistant"):
        print(f"Analysis: {last_msg.text.value}")
    
    # Cleanup by deleting the agent
    client.agents.delete_agent(agent.id)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")